This program compares the ZCTA-tract crosswalk with the EJ40 database to generate a percentage of tracts within a zip code that are identified EJ communities. 

This is not a perfect approximation for the percentage of people in a zip code living in EJ communities; while census tracts are typically similar in population, they can vary from between 1200 and 8000 people, and some census tracts are split across zip codes.

In [21]:
import pandas

crosswalk = r'/Users/robert/Downloads/_Python/EJ40/ZCTA2tract.csv'
ej40table = r'/Users/robert/Downloads/_Python/EJ40/communities-2022-05-31-1915GMT.csv'

outpath = r'/Users/robert/Downloads/_Python/EJ40/ZCTA_EJ.csv'

lookup = pandas.read_csv(crosswalk)
ej = pandas.read_csv(ej40table)

In [29]:
#remove census tracts which don't have an associated zip code (eg, national parks)
lookup = lookup.dropna()

#keep only the columns we need: ZCTA and tract for the lookup, tract ID and if it's EJ from the EJ40 table
lookup = lookup[['NAMELSAD_ZCTA5_20', 'GEOID_TRACT_20']]
ej = ej[['Census tract ID','Identified as disadvantaged']]

,Census tract ID,Identified as disadvantaged
0,1001020100,False
1,1001020200,False
2,1001020300,False
3,1001020400,False
4,1001020500,False
...,...,...
74129,78030960900,False
74130,78030961000,False
74131,78030961100,False
74132,78030961200,True


In [30]:
#format the zip codes nicely and set census tract IDs as the indices for the ej table - only run this block ONCE
for i, row in lookup.iterrows():
    lookup.at[i, 'NAMELSAD_ZCTA5_20'] = row[0][6:]
    
ej.set_index('Census tract ID',inplace=True)

,Identified as disadvantaged
Census tract ID,
1001020100,False
1001020200,False
1001020300,False
1001020400,False
1001020500,False
...,...
78030960900,False
78030961000,False
78030961100,False


In [44]:
joined = pandas.DataFrame(columns=['zip', 'num_tracts', 'percent_ej'])
tracts_not_found = []

prev_zip = 0
for i, row in lookup.iterrows():
    this_zip = row[0]
    try:
        ej_int = int(ej.loc[row[1]]) * 100 #100 if the tract is an EJ community, 0 otherwise
        if this_zip == prev_zip:
            current_ind = len(joined) - 1
            num_tracts = joined.at[current_ind, 'num_tracts']
            print()
            joined.at[current_ind, 'num_tracts'] = num_tracts + 1
            percent_calc = ((joined.at[current_ind, 'percent_ej'] * num_tracts) + ej_int) / (num_tracts + 1)
            joined.at[current_ind, 'percent_ej'] = percent_calc
        else:
            joined.loc[len(joined)] = [this_zip, 1, ej_int]
        prev_zip = this_zip
    except:
        tracts_not_found.append(row[1])

In [45]:
joined.to_csv(outpath)